In [2]:
import urllib.request, urllib.parse, urllib.error
import json

In [6]:
def print_json(input_obj,level=0):

    if isinstance(input_obj,dict) == False:
        tab=""
        for x in range(level):
            tab += "\t"
        print(f"{tab}{input_obj}")
        return
   #//*** If there is a valid Poster URL and Title.
    #//*** Download and Display the movie poster
    if "Poster" in input_obj.keys() and "Title" in input_obj.keys():
        poster_url = input_obj["Poster"]
        poster_name = input_obj["Title"]
        
        try:
            get_poster(poster_name,poster_url)
        except:
            print("[No Poster Available]")

    #//*** Tabs are used for recursion
    #//*** For every level of recursion, add a tab
    #//*** My test data only goes down to two levels, it will be fun to test with more
    
    tab=""
    for x in range(level):
        tab += "\t"
    
    #//*** Loop through the JSON keys and values
    for key,value in input_obj.items():
        
        #//*** If the response is True, skip the display
        #//*** This will print on a fale response
        if key == "Response" and value == "True":
            continue
        
        #//*** Skip Poster, it should already by handled
        if key == "Poster":
            continue
    
        #//*** If the Value is a string, print the key and value.
        if isinstance(value,str) or isinstance(value,int):
            print(f"{tab}{key}: {value}")

        #//*** If the Object is a list, print the key
        #//*** Iterate through the list calling print_json on each item with a level + 1.
        #//*** Which will increase the tab level with each call ---- Fun Stuff
        elif isinstance(value,list):
                print(f"{tab}{key}:")
                for item in value:
                    print_json(item,level+1)
        elif isinstance(value,dict):
            print(f"{tab}{key}:")
            print_json(value,level+1)            
        else:
            #//*** If it's not a list or string, print it anyway. This leaves a future debugging path open
            print(f"{tab}-->{key}: {type(value)} {value}")

In [33]:
request_url = "https://www.zillow.com/homes/San-Francisco,-CA_rb/"

response = urllib.request.urlopen(request_url)
print(response.read().decode())

<html><head><meta name="robots" content="noindex, nofollow"/><link href="https://www.zillowstatic.com/vstatic/80d5e73/static/css/z-pages/captcha.css" type="text/css" rel="stylesheet" media="screen"/><script>
        window._pxAppId = 'PXHYx10rg3';
        window._pxJsClientSrc = '/HYx10rg3/init.js';
        window._pxHostUrl = '/HYx10rg3/xhr';
        window._pxFirstPartyEnabled = true;
        window._pxreCaptchaTheme='light';
    </script><script type="text/javascript" src="https://captcha.px-cdn.net/PXHYx10rg3/captcha.js?a=c&amp;m=0"></script>
    <script>
        function getQueryString(name, url) {
            if (!url) url = window.location.href;
            name = name.replace(/[\[\]]/g, "\\$&");
            var regex = new RegExp("[?&]" + name + "(=([^&#]*)|&|#|$)"),
                results = regex.exec(url);
            if (!results) return null;
            if (!results[2]) return '';
            return decodeURIComponent(results[2].replace(/\+/g, " "));
        }
        doc